# <font color=slateblue>**Importing Neccessary Libraries**</font> #

In [ ]:
import zipfile
from sklearn.preprocessing import Normalizer
from google.colab import drive
import numpy as np
from PIL import Image, ImageDraw
from numpy import linalg as LA
import os, fnmatch, glob
from sklearn.metrics import confusion_matrix
from sklearn.decomposition import PCA
import random
import matplotlib.image as mpImg
import os
import pandas as pd
from glob import glob   
from matplotlib import pyplot as plt 
import cv2
from sklearn.preprocessing import StandardScaler

# <font color=slateblue>**Neccessary Functions**</font> #

In [ ]:
def batch(iterable, n=1):
    l = len(iterable)
    for ndx in range(0, l, n):
        yield iterable[ndx:min(ndx + n, l)]

In [ ]:
def parseAgeSexEthnicity(address):
  splited = address.split('_')
  age = splited[0]
  age = age.replace('/content/drive/MyDrive/DeepNew/DataSetUnzip/UTKFace/', '')
  sex = splited[1]
  ethnicity = splited[2]
  validCaption = (age.isdigit() and sex.isdigit() and ethnicity.isdigit())
  return age, sex, ethnicity, validCaption

In [ ]:
def loadImagesDirectory(directories, size = -1):
  _image = []; _age = [];  _sex = []
  _ethnicity = []
  addr_files = directories
  for addr in addr_files:
    age, sex, ethnicity, validCaption = parseAgeSexEthnicity(address = addr)
    if validCaption:
      _age.append(int(age))
      _sex.append(int(sex))
      _ethnicity.append(int(ethnicity))
      tmp_image = cv2.imread(addr)
      _image.append(np.asarray(tmp_image))
    else:
      print('Age: ', age, 'Sex: ', sex, 'Ethnicity: ', ethnicity)
    if len(_image) == size:
      break
  dataR = {'Image':np.asarray(_image), 'Age': np.asarray(_age), 'Sex':np.asarray(_sex), 'Ethnicity': np.asarray(_ethnicity)}
  return dataR

In [ ]:
def flatten(dataSet):
  dataSet = np.asarray(dataSet)
  shapeData = np.shape(dataSet)
  dim = 1
  for i in range(1, len(shapeData)): dim = dim*shapeData[i]
  dataSetFlatten = dataSet.reshape(shapeData[0], dim )
  return dataSetFlatten

In [ ]:
def pcaNormalizer_directory(n_comp, dataDirectory, sizeData):
  addr_files = sorted(glob(dataDirectory))
  random.shuffle(addr_files)
  dataSet = loadImagesDirectory(directories = addr_files, size = sizeData) 
  dataSet = dataSet['Image']
  dataSetFlatten = flatten(dataSetImg)
  print("**** ",np.shape(dataSetFlatten[0:2]))
  #normalizer = Normalizer().fit(dataSetFlatten[0:1000]) 
  normalizer = StandardScaler().fit(dataSetFlatten)
  dataSetFlatten = normalizer.transform(dataSetFlatten)
  pcaTrans = PCA(n_components = n_comp)
  pcaTrans.fit(dataSetFlatten) 
  return  pcaTrans, normalizer

In [ ]:
def prep_data(dataDirectory , pcaTransform, segmentSize, normalizerTrans):
  addr_files = sorted(glob(dataDirectory))
  random.shuffle(addr_files)
  cnt = 0
  for addr_files_batch in batch(addr_files, segmentSize):
    dataSet = loadImagesDirectory(directories = addr_files_batch)
    dataSetImg = dataSet['Image']
    dataSetFlatten = flatten(dataSetImg)
    dataSetFlatten = normalizerTrans.transform(dataSetFlatten)
    transformedData = pcaTransform.transform(dataSetFlatten)
    df = pd.DataFrame(np.asarray(transformedData))
    df['Age'] = dataSet['Age']
    df['Sex'] = dataSet['Sex']
    df['Ethnicity'] = dataSet['Ethnicity']
    df.to_csv(r'/content/drive/MyDrive/DeepNew/RedDimData7/Segment{0}.csv'.format(cnt), index = False)
    cnt +=1

# <font color=slateblue>**Train Net and Test that**</font> #

In [ ]:
addr_filesI =  '/content/drive/MyDrive/DeepNew/DataSetUnzip/UTKFace/*.jpg'
pcaTrans, normalizer = pcaNormalizer_directory(n_comp = 128, dataDirectory = addr_filesI, sizeData = 4000)

In [ ]:
prep_data(dataDirectory = addr_filesI , pcaTransform = pcaTrans, segmentSize = 1000, normalizerTrans = normalizer)

Age:  39 Sex:  1 Ethnicity:  20170116174525125.jpg.chip.jpg
Age:  61 Sex:  1 Ethnicity:  20170109150557335.jpg.chip.jpg
Age:  61 Sex:  1 Ethnicity:  20170109142408075.jpg.chip.jpg
